# VGG19 model

I have tried to implement VGG19 architecture on the CIFAR-10 dataset. I resized the images to 224*224 to match the Imagenet dataset. The architecture is kept same and some hyperparameters are also kept same. The batch size used is 20 instead of original 256.

In [2]:
import torch
import numpy as np

In [3]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [4]:
torch.cuda.empty_cache()

In [5]:
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 20
# percentage of training set to use as validation
valid_size = 0.2

# convert data to a normalized torch.FloatTensor
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),transforms.Resize((224,224))])

# choose the training and test datasets
train_data = datasets.CIFAR10('data', train=True,
                              download=True, transform=transform)
test_data = datasets.CIFAR10('data', train=False,
                             download=True, transform=transform)

# obtain training indices that will be used for validation
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)

# specify the image classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']


Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


The Architecture

In [7]:
import torch.nn as nn
import torch.nn.functional as F

class net(nn.Module):
  def __init__(self):
    super(net,self).__init__()
    self.conv1=nn.Conv2d(3,64,3,padding=1)
    self.conv2=nn.Conv2d(64,64,3,padding=1)
    #maxpool
    self.conv3=nn.Conv2d(64,128,3,padding=1)
    self.conv4=nn.Conv2d(128,128,3,padding=1)
    #maxpool
    self.conv5=nn.Conv2d(128,256,3,padding=1)
    self.conv6=nn.Conv2d(256,256,3,padding=1)#*3
    #maxpool
    self.conv7=nn.Conv2d(256,512,3,padding=1)
    self.conv8=nn.Conv2d(512,512,3,padding=1)#*3#maxpool#*4
    #maxpool
    self.pool=nn.MaxPool2d(2,2)
    self.fc1=nn.Linear(7*7*512,4096)
    self.fc2=nn.Linear(4096,512)
    self.fc3=nn.Linear(512,10)

    self.dropout=nn.Dropout(p=0.5)

  def forward(self,x):
    x=F.relu(self.conv1(x))
    x=F.relu(self.conv2(x))
    x=self.pool(x)
    x=F.relu(self.conv3(x))
    x=F.relu(self.conv4(x))
    x=self.pool(x)
    x=F.relu(self.conv5(x))
    x=F.relu(self.conv6(x))
    x=F.relu(self.conv6(x))
    x=F.relu(self.conv6(x))
    x=self.pool(x)
    x=F.relu(self.conv7(x))
    x=F.relu(self.conv8(x))
    x=F.relu(self.conv8(x))
    x=F.relu(self.conv8(x))
    x=self.pool(x)
    x=F.relu(self.conv8(x))
    x=F.relu(self.conv8(x))
    x=F.relu(self.conv8(x))
    x=F.relu(self.conv8(x))
    x=self.pool(x)
    x=torch.flatten(x,1)
    x=self.dropout(F.relu(self.fc1(x)))
    x=self.dropout(F.relu(self.fc2(x)))
    x=self.fc3(x)
    x=F.softmax(x,dim=1)

    return x

In [8]:
model=net()
print(model)
if train_on_gpu:
    model.cuda()

net(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv7): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv8): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=25088, out_features=4096, bias=True)
  (fc2): Linear(in_features=4096, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=10, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [16]:
import torch.optim as optim
from  torch.optim.lr_scheduler import ReduceLROnPlateau
criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(),lr=0.001,momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, 'min',patience=5)

Attempt to train the model at learning rate of 0.001. It did not get completed.

In [ ]:
epoch=15

valid_loss_min = np.Inf # track change in validation loss

for e in range(epoch):

    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    for data, target in train_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
        #print(loss.item()*data.size(0))
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for data, target in valid_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)
    
    # calculate average losses
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)

    scheduler.step(valid_loss)
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        e, train_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_cifar.pt')
        valid_loss_min = valid_loss

Epoch: 0 	Training Loss: 1.842069 	Validation Loss: 0.460523
Validation loss decreased (inf --> 0.460523).  Saving model ...
Epoch: 1 	Training Loss: 1.842065 	Validation Loss: 0.460525
Epoch: 2 	Training Loss: 1.842064 	Validation Loss: 0.460527
Epoch: 3 	Training Loss: 1.842065 	Validation Loss: 0.460528
Epoch: 4 	Training Loss: 1.842062 	Validation Loss: 0.460530
Epoch: 5 	Training Loss: 1.842059 	Validation Loss: 0.460532
Epoch: 6 	Training Loss: 1.842058 	Validation Loss: 0.460534


In [9]:
import torch.optim as optim
from  torch.optim.lr_scheduler import ReduceLROnPlateau
criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(),lr=0.1,momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, 'min',patience=5)

Attempt to train model at lr of 0.1.

In [11]:
epoch=10
valid_loss_min = np.Inf
for e in range(1,epoch+1):

    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    for data, target in train_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
        #print(loss.item()*data.size(0))
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for data, target in valid_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)
    
    # calculate average losses
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)

    scheduler.step(valid_loss)
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        e, train_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_cifar.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 1.842208 	Validation Loss: 0.460548
Validation loss decreased (inf --> 0.460548).  Saving model ...
Epoch: 2 	Training Loss: 1.842204 	Validation Loss: 0.460541
Validation loss decreased (0.460548 --> 0.460541).  Saving model ...
Epoch: 3 	Training Loss: 1.842185 	Validation Loss: 0.460562
Epoch: 4 	Training Loss: 1.842207 	Validation Loss: 0.460551
Epoch: 5 	Training Loss: 1.842188 	Validation Loss: 0.460560
Epoch: 6 	Training Loss: 1.842169 	Validation Loss: 0.460572
Epoch: 7 	Training Loss: 1.842181 	Validation Loss: 0.460535
Validation loss decreased (0.460541 --> 0.460535).  Saving model ...
Epoch: 8 	Training Loss: 1.842107 	Validation Loss: 0.460538
Epoch: 9 	Training Loss: 1.842095 	Validation Loss: 0.460541
Epoch: 10 	Training Loss: 1.842085 	Validation Loss: 0.460543


# Model Summary

In [12]:
from torchsummary import summary
summary(model,(3,224,224),20)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [20, 64, 224, 224]           1,792
            Conv2d-2         [20, 64, 224, 224]          36,928
         MaxPool2d-3         [20, 64, 112, 112]               0
            Conv2d-4        [20, 128, 112, 112]          73,856
            Conv2d-5        [20, 128, 112, 112]         147,584
         MaxPool2d-6          [20, 128, 56, 56]               0
            Conv2d-7          [20, 256, 56, 56]         295,168
            Conv2d-8          [20, 256, 56, 56]         590,080
            Conv2d-9          [20, 256, 56, 56]         590,080
           Conv2d-10          [20, 256, 56, 56]         590,080
        MaxPool2d-11          [20, 256, 28, 28]               0
           Conv2d-12          [20, 512, 28, 28]       1,180,160
           Conv2d-13          [20, 512, 28, 28]       2,359,808
           Conv2d-14          [20, 512,

In [13]:
model.load_state_dict(torch.load('model_cifar.pt'))

<All keys matched successfully>